# 여러 문서에서 찾아서 답변하는 챗봇 만들기

> 유튜브 [빵형의 개발도상국]

- QA ChatBot
- LangChain
- ChatGPT (gpt-3.5-turbo)
- ChromaDB

> Reference: https://youtu.be/3yPBVii7Ct0

In [ ]:
!pip install -q langchain openai tiktoken chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.8/220.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.4/502.4 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Setting up LangChain

OpenAI API Key

https://platform.openai.com/account/api-keys

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "#"

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader

## Load multiple and process documents

In [ ]:
loader = DirectoryLoader('./data', glob="*.txt", loader_cls=TextLoader)

documents = loader.load()

len(documents)

4

In [ ]:
documents

[Document(page_content='"삼성바이오로직스는 생명과학 분야에서의 선도적인 위치와 신약 개발에 대한 투자로 인해 향후 성장이 예상됩니다. 안정적인 수익성이 투자자에게 안정감을 제공합니다.\nSK텔레콤은 5G 기술의 선도적인 보유와 통신 시장에서의 강세로 인해 향후 성장 가능성이 높다고 판단하여 투자합니다.\n네이버는 온라인 플랫폼의 선도적인 입지와 다양한 서비스로 향후 안정적인 수익이 기대됩니다. 디지털 콘텐츠와 플랫폼 사업으로 인한 수익 상승이 주가에 긍정적인 영향을 미칠 것으로 예상됩니다."\n', metadata={'source': 'data/2.txt'}),
 Document(page_content='"삼성전자는 글로벌 반도체 시장에서의 선도적인 입지와 안정적인 실적으로 안정적인 투자 대상입니다. 향후 반도체 수요 증가로 인해 수익성이 높아질 것으로 예상됩니다.\nLG는 다양한 사업 분야에서의 경험과 혁신적인 기술력으로 주가 상승의 기회를 제공할 것으로 기대됩니다. 특히, LG의 전자제품과 디스플레이 기술에 대한 수요가 지속적으로 증가하고 있습니다.현대자동차는 글로벌 자동차 시장에서의 강세와 친환경 차량에 대한 수요 증가로 인해 향후 성장이 예상됩니다. 다양한 차종과 지속적인 연구 개발로 시장에서의 경쟁 우위를 유지하고 있습니다."\n\n', metadata={'source': 'data/1.txt'}),
 Document(page_content='친환경 자동차 시장에서의 선도적인 입지와 혁신적인 기술로 주가 상승이 기대됩니다. 향후 전기차 수요 증가와 함께 기업의 성장이 예상됩니다.\n생명과학 분야에서의 선도적인 위치와 신약 개발에 대한 투자로 인해 향후 성장이 예상됩니다. 안정적인 수익성이 투자자에게 안정감을 제공합니다.\n글로벌 스트리밍 서비스 시장에서의 지배적인 지위와 콘텐츠 풍부성으로 투자가치가 높아보입니다. 디지털 콘텐츠 산업의 성장이 지속될 것으로 예상됩니다.', metadata={'source': 'data/3.txt'})

## Split texts

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

len(texts)

4

## Create Chroma DB

1. Text -> Embbedings
2. `db` 폴더에 데이터 저장
3. DB 초기화
4. `db` 폴더로부터 DB 로드

In [ ]:
persist_directory = 'db'

embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embedding,
    persist_directory=persist_directory)

In [ ]:
# db 초기화 -> 처음에만 사용해주면 됌
vectordb.persist()
vectordb = None

In [ ]:
# db load
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding)

## Make a retriever

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
docs = retriever.get_relevant_documents("삼성전자는 지금 상황이 어떤 것 같애?")

for doc in docs:
    print(doc.metadata["source"])

data/1.txt
data/2.txt
data/3.txt
data/4.txt


### 결과를 k개 반환

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [ ]:
docs = retriever.get_relevant_documents("삼성전자는 지금 상황이 어떤 것 같애?")

for doc in docs:
    print(doc.metadata["source"])

data/1.txt
data/2.txt
data/3.txt


## Make a chain

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True)

In [ ]:
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

## Query

In [ ]:
query = "삼성전자는 지금 상황이 어떤 것 같애?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 삼성전자는 글로벌 반도체 시장에서의 선도적인 입지와 안정적인 실적으로 안정적인 투자 대상으로 평가되고 있으며, 향후 반도체 수요 증가로 인해 수익성이 높아질 것으로 예상됩니다.


Sources:
data/1.txt
data/2.txt
data/3.txt


In [ ]:
llm_response

{'query': '삼성전자는 지금 상황이 어떤 것 같애?',
 'result': ' 삼성전자는 글로벌 반도체 시장에서의 선도적인 입지와 안정적인 실적으로 안정적인 투자 대상으로 평가되고 있으며, 향후 반도체 수요 증가로 인해 수익성이 높아질 것으로 예상됩니다.',
 'source_documents': [Document(page_content='"삼성전자는 글로벌 반도체 시장에서의 선도적인 입지와 안정적인 실적으로 안정적인 투자 대상입니다. 향후 반도체 수요 증가로 인해 수익성이 높아질 것으로 예상됩니다.\nLG는 다양한 사업 분야에서의 경험과 혁신적인 기술력으로 주가 상승의 기회를 제공할 것으로 기대됩니다. 특히, LG의 전자제품과 디스플레이 기술에 대한 수요가 지속적으로 증가하고 있습니다.현대자동차는 글로벌 자동차 시장에서의 강세와 친환경 차량에 대한 수요 증가로 인해 향후 성장이 예상됩니다. 다양한 차종과 지속적인 연구 개발로 시장에서의 경쟁 우위를 유지하고 있습니다."', metadata={'source': 'data/1.txt'}),
  Document(page_content='"삼성바이오로직스는 생명과학 분야에서의 선도적인 위치와 신약 개발에 대한 투자로 인해 향후 성장이 예상됩니다. 안정적인 수익성이 투자자에게 안정감을 제공합니다.\nSK텔레콤은 5G 기술의 선도적인 보유와 통신 시장에서의 강세로 인해 향후 성장 가능성이 높다고 판단하여 투자합니다.\n네이버는 온라인 플랫폼의 선도적인 입지와 다양한 서비스로 향후 안정적인 수익이 기대됩니다. 디지털 콘텐츠와 플랫폼 사업으로 인한 수익 상승이 주가에 긍정적인 영향을 미칠 것으로 예상됩니다."', metadata={'source': 'data/2.txt'}),
  Document(page_content='친환경 자동차 시장에서의 선도적인 입지와 혁신적인 기술로 주가 상승이 기대됩니다. 향후 전기차 수요 증가와 함께 기업의 성장이 예상됩니다.\n생명과학 분야에서의 선도적인 위치와 신약 개발에 대한 투자로

In [ ]:
query = "요새 자동차 시장은 어때?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 향후 전기차 수요 증가와 함께 기업의 성장이 예상되며, 현대자동차는 글로벌 자동차 시장에서의 강세와 친환경 차량에 대한 수요 증가로 인해 향후 성장이 예상됩니다.


Sources:
data/3.txt
data/1.txt
data/2.txt


# 블러 처리

In [ ]:
pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 33.5 MB/s eta 0:00:00


In [ ]:
from konlpy.tag import Okt
from transformers import BertTokenizer, BertForTokenClassification
import torch

# KoBERT 모델 및 토크나이저를 로드합니다.
model_name = "monologg/kobert"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForTokenClassification.from_pretrained(model_name)

# KoNLPy에서 형태소 분석기를 생성합니다.
okt = Okt()

# 텍스트에서 명사와 동사를 추출하고 해당 부분을 "-"로 대체하는 함수를 정의합니다.
def replace_nouns_and_verbs_with_dash(text):
    # 형태소 분석을 통해 명사와 동사를 추출합니다.
    morphemes = okt.pos(text)
    nouns_and_verbs = [(word, pos) for word, pos in morphemes if pos in ['Noun', 'Verb']]

    # 문장을 여러 부분으로 나누어서 처리합니다.
    chunk_size = 10
    chunks = [nouns_and_verbs[i:i+chunk_size] for i in range(0, len(nouns_and_verbs), chunk_size)]

    # 명사와 동사만 추출하고 해당 부분을 "-"로 대체합니다.
    result_text = text
    for chunk in chunks:
        inputs = tokenizer(" ".join([word for word, pos in chunk]), return_tensors="pt", truncation=True, padding=True)
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=2)

        for i, (word, pos) in enumerate(chunk):
            if pos in ['Noun', 'Verb']:
                if predictions[0][i].item() in [1, 3]:  # 명사 또는 동사 토큰인 경우
                    result_text = result_text.replace(word, "-")

    return result_text

# 예시 텍스트를 정의합니다.
sample_text = "삼성전자는 글로벌 반도체 시장에서의 선도적인 입지와 안정적인 실적으로 안정적인 투자 대상으로 평가되고 있으며, 향후 반도체 수요 증가로 인해 수익성이 높아질 것으로 예상됩니다."

# 명사와 동사가 있는 부분을 "-"로 대체한 텍스트를 얻습니다.
result_text = replace_nouns_and_verbs_with_dash(sample_text)

# 결과를 출력합니다.
print(result_text)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


삼성-는 - - -에서의 선도적인 입지와 안정적인 실적으로 안정적인 투자 대상으로 -- 있으며, - - - -로 인해 수익성이 높아질 것으로 예상-.
